# Gap (prelaunch) 0.9 - July 2018
## NLP and CV Data Engineering Framework

<b>[Github](https://github.com/andrewferlitsch/gap)</b>

# Document Preparation for NLP

## Applications (Open Source) Recommended

1. Artifex's Ghostscript - extracting text from text PDF
2. ImageMagic's Magick - extracting image from scanned PDF
3. Google's Tesseract - OCR of scanned/image captured text
4. NLTK Toolkit - Syntax Analysis/Parts of Speech

### Github Account

https://github.com/andrewferlitsch/epipog-nlp

### Ghostscript

1. Download link : https://www.ghostscript.com/download/gsdnld.html
        
    Use the Free Version<br/>

    I have a 64bit Windows laptop, so I am using this version: Ghostscript 9.23 for Windows (64 bit).<br/><br/>

2. Check if path to the program is in your PATH variable. 

    A. Open a command shell.<br/>
    B. Type gswin64c in the command line.<br/>
    C. If not found, add it to your path variable. For me, it is: C:\Program Files\gs\gs9.23\bin<br/>




### Magick

1. Download Link: https://www.imagemagick.org/script/download.php

    Use the static version (dynamic is for DLL inclusion).<br/>
    
    Use the 8bits per pixel version.<br/>
    
    I have a 64bit Windows laptop, so I am using this version: ImageMagick-7.0.8-1-Q8-x64-static.exe<br/><br/>
    
2. Check if path to the program is in your PATH variable.
 
    A. Open a command shell.<br/>
    B. Type magick in the command line.<br/>
    C. If not found, add it to your path variable. For me, it is: C:\Program Files\ImageMagic-7.0.8-Q8

### Tesseract

1. Download Link: https://github.com/tesseract-ocr/tesseract/wiki/Downloads

    A. Make sure to add the English Language training data to the tessdata subdirectory where tesseract is installed.<br/><br/>

2. Check if path to program is in your PATH variable:

    A. Open a command shell.<br/>
    B. Type tesseract in the command line.<br/>
    C. If not found, add it your path variable. For me, it is C:\Program Files\tesseract-Win64\<br/>
    
3. Install the English Training Data files as: C:\Program Files\tesseract-Win64\tessdata

    You can get a copy from my github account.

### NLTK Toolkit

1. Documentation: http://www.nltk.org/


2. Install NLTK using pip:

    pip install nltk


### Ghostscript Example: Extracting Text from Text PDF

Let's try some examples with using Ghostscript and PDF documents.

First, let's get the number of pages in the PDF. (yes, ghostscript's options are somewhat cryptic). We will do it on a 140 page file.

In [ ]:
import os
os.chdir("../")
!cd
#!ls #on linux

In [ ]:
#use !gs on Linux
!gswin64c -dBATCH -q -dNODISPLAY -c "("tests/plan/afspa.pdf") (r) file runpdfbegin pdfpagecount = quit"

Let's now try to split a PDF into individual pages. In the command below, we tell Ghostscript to split out page 1. We could
do this in a for loop and extract each page one at a time.

Note that we set the output DEVICE to pdfwrite. This is the command to Ghostscript to output a PDF file.

In [ ]:
!gswin64c -dBATCH -dNOPAUSE -sOutputFile="tests/plan/afspa1.pdf" -sPageList=1 -sDEVICE=pdfwrite "tests/plan/afspa.pdf"

Let's look at what is inside of the first PDF page. Is it text, a scanned image, a mix of text and images? We can guessimate this by looking at the PDF Resource directive inside of the PDF file.

    /Text   -> Text
    /ImageB -> B&W Image
    /ImageC -> Color Image
    /ImageI -> Indexed Image

We will do this using Epipog's PDFResource object. We will also set the debug flag to output the version and PDF resources found to the standard output.

In [ ]:
# Import Epipog PDFResource class from the pdf_res module
from gapml.pdf_res import PDFResource

In [ ]:
res = PDFResource("tests/plan/afspa1.pdf", debug=True)

The PDFResource properties text and image will tell us if the PDF file contains text and/or images respectively.

In [ ]:
# Let's now check whether the page is a text PDF, scanned PDF, or mix text and images.
print(res.text)
print(res.image)

Let's now extract the text from this single page PDF file using Ghostscript.

Note that we set the output DEVICE to txtwrite. This is the command to Ghostscript to output a PDF file.

In [ ]:
!gswin64c -dBATCH -dNOPAUSE -sOutputFile="tests/plan/afspa1.txt" -sPageList=1 -sDEVICE=txtwrite "tests/plan/afspa1.pdf"

In [ ]:
# use !ls on linux
!dir tests\\plan\\afspa*.pdf

Let's try another PDF document, which will have an unexpected result - which I will explain.

In [ ]:
# Split page 1 from the PDF file
!gswin64c -dBATCH -dNOPAUSE -sOutputFile="tests/plan/il1.pdf" -sPageList=1 -sDEVICE=pdfwrite "tests/plan/il.pdf"

# Extract the text from the PDF page
!gswin64c -dBATCH -dNOPAUSE -sOutputFile="tests/plan/il1.txt" -sPageList=1 -sDEVICE=txtwrite "tests/plan/il1.pdf"

Let's view the contents of the extracted text file.

OMG. It's just a lot of unprintable ASCII control characters. What happened?

The PDF Resource directive is just a hint. It doesn't mean that it is correct. So, in this case, this is really a scanned PDF.

In [ ]:
!notepad.exe tests\\plan\\il1.txt

### Ghostscript/Tesseract Example:  Extract Text from Scanned PDF example

So the il.pdf file appears to be a scanned PDF. So, let's extract the scanned page as a PNG image using Ghostscript.

This time, we will set the output device to a grayscale PNG image. Ghostscript actually renders an image (vs. merely extract). This gives us an opportunity to tell Ghostscript the resolution of the generated image, which will affect the OCR quality. I good rule of thumb is 300dpi. I've found 72 and 150dpi give poor OCR, 200 is okay on many things, but 300 generally is good for all cases.


In [ ]:
# Extract the scanned image from the PDF page
!gswin64c -dBATCH -dNOPAUSE -sOutputFile="tests/plan/il1.png" -sPageList=1 -sDEVICE=pnggray  -r300 "tests/plan/il1.pdf"

Now we will use Tesseract to extract the text from the PNG image.

In [ ]:
!tesseract "tests/plan/il1.png" "tests/plan/il1"

In [ ]:
import matplotlib.pyplot as plt

# This line is specific to python notebooks (not python). 
# It causes plots to automatically be rendered (displayed) without issuing a show command.
%matplotlib inline

# Importing OpenCV
import cv2
image = cv2.imread("tests/plan/il1.png", 0)
plt.imshow(image)

In [ ]:
#clean dir
os.remove('tests/plan/afspa1.pdf')
os.remove('tests/plan/afspa1.txt')

os.remove('tests/plan/il1.pdf')
os.remove('tests/plan/il1.png')
os.remove('tests/plan/il1.txt')

## END OF SESSION 0